# Structured Output with Google Gemini
In order to build reliable pipelines in which LLMs consistent return output in the same format, we are using a **Structured Output**
- This means that we define a blueprint for the output
- We pass the 'blueprint' to the LLM
- Then the LLM output will confirm to the blueprint.

This 'blueprint' in the LLM jargon is often called a "schema".

**To generate structured outputs with Anthropic, we'll use the library `instructor`**.
This will make it very easy to switch between different model providers.

## Setup Google Gemini API

In [1]:
!pip install instructor --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 2.0 MB/s eta 0:00:00


In [2]:
import google.generativeai as genai
import os
from pydantic import BaseModel
from enum import Enum
import instructor

In [3]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

In [4]:
genai.configure(api_key=GOOGLE_API_KEY)

## Generate JSON
When the model is configured to output JSON, it responds to any prompt with JSON-formatted output.

You can control the structure of the JSON response by supplying a schema. There are two ways to supply a schema to the model:

- As text in the prompt
- As a structured schema supplied through model configuration

Both approaches work in both Gemini 1.5 Flash and Gemini 1.5 Pro.

In [5]:
class City(str, Enum):
    aria = "Aria"
    kniga = "Kniga"
    aquabah = "Aquabah"
    torini = "Torini"

class Character(BaseModel):
    name: str
    age: int
    city: City
    job: str
    two_sentences_background_story: str
    inventory: list[str]

In [6]:
google_client = genai.GenerativeModel(
    model_name="gemini-1.5-flash")

In [7]:
client = instructor.from_gemini(
    client=google_client,
    mode=instructor.Mode.GEMINI_JSON,
)

In [8]:
# note that client.chat.completions.create will also work
player = client.messages.create(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful table-top RPG gamemaster assistant."},
        {
            "role": "user",
            "content": "Generate an Archer character from Kniga for a low-fantasy RPG campaign. Be creative"},
    ],
    response_model=Character,
)

In [9]:
player

Character(name='Liliana', age=26, city=<City.kniga: 'Kniga'>, job='Archer', two_sentences_background_story='Liliana grew up in the forests outside Kniga, learning to hunt from a young age. After her family was killed by bandits, she sought refuge in the city, using her skills to survive.', inventory=['Longbow', 'Quiver of arrows', 'Hunting knife', 'Leather jerkin', 'Waterskin', 'Small pouch of coins'])

Learn more about instructor here: https://python.useinstructor.com/